# Step 3: Extract Entities

Extract entities from parsed documents using Foundation Model.

**Pipeline:** batch_read  
**Step:** extract_entities  
**Service:** entity_extractor_default

**Inputs:**
- Source: Parsed documents from `parse_document` step
- Column: extracted_text

**Outputs:**
- Table: `${silver_schema}.batch_read_entities`
- Extracted entities in JSON format


In [ ]:
# Imports
from pyspark.sql import functions as F
from pyspark.sql.types import *
import json

# Load configuration from task variables (set by setup task)
config_json = dbutils.jobs.taskValues.get(taskKey="setup", key="config")
config = json.loads(config_json)

print("✅ Configuration loaded from task variables")


In [ ]:
# Get step configuration
pipeline_config = config["pipelines"]["batch_read"]
step_config = pipeline_config["pdf"]["standard"]["steps"][2]  # extract_entities step (index 2, after read and parse)

service_name = step_config["service"]
service_config = config["services"][service_name]

source_table = step_config["source"]  # Input from previous step (parse_document)
source_column = step_config["source_column"]  # Column to extract from
output_table = step_config["output"]
prompt_template = step_config["prompt_template"]

print(f"Service: {service_name}")
print(f"Source Table: {source_table}")
print(f"Source Column: {source_column}")
print(f"Output Table: {output_table}")


In [ ]:
# Read from previous step output table
df_documents = spark.table(source_table)

print(f"Documents to process: {df_documents.count()}")
df_documents.printSchema()


In [ ]:
# Call Foundation Model for entity extraction
# TODO: Replace with actual Model Serving API call

def extract_entities_udf(text):
    """
    Call foundation model to extract entities.
    TODO: Implement actual API call to Model Serving endpoint
    """
    # Placeholder logic
    return json.dumps({
        "document_type": "unknown",
        "key_entities": [],
        "summary": text[:200] if text else "",
        "metadata": {}
    })

# Register UDF
from pyspark.sql.types import StringType
extract_udf = F.udf(extract_entities_udf, StringType())

# Apply entity extraction
df_entities = (
    df_documents
    .withColumn("extracted_entities", extract_udf(F.col(source_column)))
    .withColumn("extraction_timestamp", F.current_timestamp())
    .withColumn("extraction_date", F.current_date())
    .select(
        "document_id",
        "file_name",
        "file_path",
        source_column,
        "extracted_entities",
        "extraction_date",
        "extraction_timestamp"
    )
)

print(f"✅ Extracted entities from {df_entities.count()} documents")


In [ ]:
# Write to Silver table
(
    df_entities.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(output_table)
)

print(f"✅ Entities written to {output_table}")
